In [ ]:
'''
load original df
'''

import pandas as pd
import os
import numpy as np
from functions import *
from scipy.stats import mannwhitneyu
import seaborn as sns
from statannotations.Annotator import Annotator
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore') #supresses warnings for now
#warnings.filterwarnings(action='once') #shows warnings once

dataFilePath = os.path.join(os.getcwd(),'Data')

kicsSVdf = pd.read_csv(os.path.join(dataFilePath,'kics_structural_variations.txt'), sep = '\t', header = 0)
#kicsTEdf = pd.read_csv(os.path.join(dataFilePath,'kics_transposable_elements.txt'), sep = '\t', header = 0)
lfsSVdf = pd.read_csv(os.path.join(dataFilePath,'lfs_structural_variations.txt'), sep = '\t', header = 0)
#lfsTEdf = pd.read_csv(os.path.join(dataFilePath,'lfs_transposable_elements.txt'), sep = '\t', header = 0)
kicsClinicdf = pd.read_csv(os.path.join(dataFilePath,'kics_clinical_main.tsv'), sep = '\t', header = 0)
lfsClinicdf = pd.read_csv(os.path.join(dataFilePath,'lfs_clinical_main.tsv'), sep = '\t', header = 0)


In [ ]:
"""
filter dfs
"""

sns.set(font_scale=1.5)

kicsSVdf = maxGDAFFilter('GD_AF', kicsSVdf)
lfsSVdf = maxGDAFFilter('GD_AF', lfsSVdf)

In [ ]:
listValSearch = ['291775', '298313', '315666', '320118', '18_1907']
# searched for kID of 63, 83, 156, 171, 219
# kID 120 141 and 232 do not have any matches...
#what would make this that much better is if it was a dict. Computationally less heavy?

for i in listValSearch: 
    kicsSVdf.drop(kicsSVdf[kicsSVdf['sample_id'].str.contains(i)==True].index, inplace = True)


In [ ]:
kicsSVF = kicsSVdf.loc[(kicsSVdf['AnnotSV type'] == 'full') & (kicsSVdf['GD_AF'] < 0.01)]
lfsSVF = lfsSVdf.loc[(lfsSVdf['AnnotSV type'] == 'full') & (lfsSVdf['GD_AF'] < 0.01)]

kicsSVF['SV chrom'] = kicsSVF['SV chrom'].astype(str)
lfsSVF['SV chrom'] = lfsSVF['SV chrom'].astype(str)

kicsSVnum = kicsSVF['sample_id'].count()
lfsSVnum = lfsSVF['sample_id'].count()


#to improve this, multi-index would be good, that way we can have less columns
mergedSVdf = pd.DataFrame(data = {'kics':kicsSVF['SV type'], 'kChrom':kicsSVF['SV chrom'], 
                                  'kId':kicsSVF['sample_id'], 'lId':lfsSVF['sample_id'],
                                  'lfs':lfsSVF['SV type'], 'lChrom':lfsSVF['SV chrom']})


In [ ]:
mergedGDAF = pd.DataFrame(data = {'kics':kicsSVF['GD_AF'],
                                'lfs':lfsSVF['GD_AF']})

mergedGDAF.replace(to_replace = [-1], value = np.nan, inplace=True)
mergedGDAF.replace(to_replace = ['-1'], value = np.nan, inplace=True)

titles = ['kics','lfs']
columns = titles

print(mergedGDAF)
boxplotPoints(titles, columns, mergedGDAF)


In [ ]:
"""
1. Graph for frequency of SV types
2. Graphs for frequency of SV types specific to chromosomes

Function to graph bar graphs
"""
    
xLabels = ['DEL','DUP','INV']
uniqueLabels = ['1','2','3','4','5','6','7','8','9','10',
                '11','12','13','14','15','16','17','18','19','20','21','22','X','Y']

deldf = mergedSVdf.loc[(mergedSVdf['kics'] == 'DEL') | (mergedSVdf['lfs']=='DEL')]
dupdf = mergedSVdf.loc[(mergedSVdf['kics'] == 'DUP') | (mergedSVdf['lfs']=='DUP')]
invdf = mergedSVdf.loc[(mergedSVdf['kics'] == 'INV') | (mergedSVdf['lfs']=='INV')]

In [ ]:
#Overall
BarGraphNormalized('kics', 'lfs', mergedSVdf, 'kics', 'lfs', 'Groups', 'Frequency/NumSamples', 
                   kicsSVnum, lfsSVnum, xLabels)

#Deletions 
BarGraphNormalized('kics', 'lfs', deldf, 'kChrom', 'lChrom', 'Chromosomes', 'Frequency/num', 
                   kicsSVnum, lfsSVnum, uniqueLabels)

#Duplicates
BarGraphNormalized('kics', 'lfs', dupdf, 'kChrom', 'lChrom', 'Chromosomes', 'Frequency/num', 
                   kicsSVnum, lfsSVnum, uniqueLabels)

#Inversions
BarGraphNormalized('kics', 'lfs', invdf, 'kChrom', 'lChrom', 'Chromosomes', 'Frequency', 
                   kicsSVnum, lfsSVnum, uniqueLabels)

In [ ]:
"""
Testing group by
"""
#probably a better way to do this

kicsSVTypedf = pd.DataFrame(data = {'kics':kicsSVF['SV type'], 
                                  'kId':kicsSVF['sample_id']})
lfsSVTypedf = pd.DataFrame(data = {'lId':lfsSVF['sample_id'],
                                  'lfs':lfsSVF['SV type']})


##kics
ktemp = kicsSVTypedf.groupby(['kId']).value_counts()

##lfs
ltemp = lfsSVTypedf.groupby(['lId']).value_counts()

#get unique identifiers

uniqueK = kicsSVTypedf['kId'].unique()
uniqueL = lfsSVTypedf['lId'].unique()

#probably a better way, but itterate through the series and get the data 

# SV Type

In [ ]:
kDelData = makeDataSet(uniqueK, ktemp, 'DEL')
lDelData = makeDataSet(uniqueL, ltemp, 'DEL')
tempDelDF = makeUnequalDF(kDelData, lDelData)

kDupData = makeDataSet(uniqueK, ktemp, 'DUP')
lDupData = makeDataSet(uniqueL, ltemp, 'DUP')
tempDupDF = makeUnequalDF(kDupData, lDupData)

kInvData = makeDataSet(uniqueK, ktemp, 'INV')
lInvData = makeDataSet(uniqueL, ltemp, 'INV')
tempInvDF = makeUnequalDF(kInvData, lInvData)

## Del 

In [ ]:
boxplotPoints(titles, columns, tempDelDF, 10, 7)

## Dup 

In [ ]:
boxplotPoints(titles, columns, tempDupDF, 10, 7)

## Inv 

In [ ]:
boxplotPoints(titles, columns, tempInvDF, 10, 7)

In [ ]:
"""
Chromosome Specific
Cleaned up
""" 


kicsMore = pd.DataFrame(data = {'SV':kicsSVF['SV type'], 'chrom':kicsSVF['SV chrom'], 
                                  'id':kicsSVF['sample_id']})
lfsMore = pd.DataFrame(data = {'id':lfsSVF['sample_id'], 'chrom':lfsSVF['SV chrom'],
                                  'SV':lfsSVF['SV type']})

numBPChrom = [249250621, 243199373, 198022430, 191154276, 180915260, 
              171115067, 159138663, 146364022, 141213431, 135534747,
              135006516, 133851895, 115169878, 107349540, 102531392,
              90354753, 81195210, 78077248, 59128983, 63025520,
              48129895, 51304566, 155270560, 59373566]

grouping = ['id', 'SV', 'chrom']



#If you have time, you should make it so that the plot can have the points showing/scattered plot



In [ ]:
"""
Normalised and Regular Graph
"""
#rename things later and put into function/i but this is the statistical test T-T
newKdf, normKdf = formatDataFrame(kicsMore, grouping, numBPChrom, uniqueK, uniqueLabels, 'DEL')

newLdf, normLdf = formatDataFrame(lfsMore, grouping, numBPChrom, uniqueL, uniqueLabels, 'DEL')
createBoxplot(newKdf, normKdf, newLdf, normLdf)


## Duplication

In [ ]:
"""
Dup
"""
newKdf, normKdf = formatDataFrame(kicsMore, grouping, numBPChrom, uniqueK, uniqueLabels, 'DUP')
newLdf, normLdf = formatDataFrame(lfsMore, grouping, numBPChrom, uniqueL, uniqueLabels, 'DUP')

createBoxplot(newKdf, normKdf, newLdf, normLdf)

### Inversion

In [ ]:
"""
INV
"""

newKdf, normKdf = formatDataFrame(kicsMore, grouping, numBPChrom, uniqueK, uniqueLabels, 'INV')
newLdf, normLdf = formatDataFrame(lfsMore, grouping, numBPChrom, uniqueL, uniqueLabels, 'INV')

createBoxplot(newKdf, normKdf, newLdf, normLdf)

In [ ]:
""" 
filter the df for CDS_length
"""

kicsSVR = kicsSVdf.loc[(kicsSVdf['AnnotSV type'] == 'split')& (kicsSVdf['GD_AF'] < 0.01) & (kicsSVdf['GD_AF'] >=0)] 
lfsSVR = lfsSVdf.loc[(lfsSVdf['AnnotSV type'] == 'split') & (lfsSVdf['GD_AF'] < 0.01) & (lfsSVdf['GD_AF'] >=0)]


mergedCDSdf = pd.DataFrame(data = {'kics':kicsSVR['CDS length'], 'lfs':lfsSVR['CDS length']})
boxplotPoints(titles, columns, mergedCDSdf, sizeV = 10)
#print(mergedCDSdf.head(50)) #they look the same